In [11]:
import pandas as pd
# import vaex
import numpy as np
import glob
import dask.dataframe as dd
import json
from sklearn.model_selection import train_test_split
import math
import csv
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import time
import _warnings
import tensorflow as tf
from tqdm import tqdm
import swifter
import argparse
import os
import sys

from importlib import reload

In [8]:
%load_ext autoreload
%autoreload 2

# sys.path.append('../')
sys.path.insert(0, os.path.dirname(os.getcwd()) + "/../")

print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/mnt/hdd2/transformer-entropy-ids/road/../', '/mnt/hdd2/transformer-entropy-ids/../', '/mnt/hdd2/transformer-entropy-ids/notebooks', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python39.zip', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9/lib-dynload', '', '/home/tiendat/miniconda3/envs/piptorchtf/lib/python3.9/site-packages', '../']


In [3]:
os.chdir(os.path.dirname(os.getcwd()) + "/road/attacks")
print(os.getcwd())

/mnt/hdd2/transformer-entropy-ids/road/attacks


In [21]:
import helper_functions

file_name = "/mnt/hdd2/transformer-entropy-ids/road/attacks/fuzzing_attack_1.log"

print(file_name)

df = helper_functions.make_can_df(file_name)
df = helper_functions.add_time_diff_per_aid_col(df)
print(df.shape)
print(df.dtypes)
display(df)

df.to_csv("/mnt/hdd2/transformer-entropy-ids/road/processed/Fuzzy_dataset.csv", index=False)

/mnt/hdd2/transformer-entropy-ids/road/attacks/fuzzing_attack_1.log
(45585, 5)
time          float64
aid             int64
data           object
ori_time      float64
time_diffs    float64
dtype: object


,time,aid,data,ori_time,time_diffs
3801,1.582211,6,0800006400000000,1.000000e+09,0.999693
6195,2.582989,6,0800006400000000,1.000000e+09,1.000778
8588,3.582923,6,0800006400000000,1.000000e+09,0.999934
10982,4.582692,6,0800006400000000,1.000000e+09,0.999769
13547,5.582793,6,0800006400000000,1.000000e+09,1.000101
...,...,...,...,...,...
48305,19.913368,1788,0000075302000040,1.000000e+09,0.099997
48545,20.014034,1788,0000075302000040,1.000000e+09,0.100666
48794,20.113337,1788,0000075302000040,1.000000e+09,0.099303
49037,20.213341,1788,0000075302000040,1.000000e+09,0.100004


In [70]:
def get_all_data(attack_dict):
    df_aggregation = []
    
    for attack_name, metadata in attack_dict.items():    
        if "masquerade" not in attack_name and "accelerator" not in attack_name and "metadata" not in attack_name:
            file_name = '{}.log'.format(attack_name)
            df_attack = helper_functions.make_can_df(file_name)
            df_attack = helper_functions.add_time_diff_per_aid_col(df_attack)
            # print(df_attack.shape)
            # print(df.dtypes)
            df_aggregation.append(df_attack)
            print(f"Finish preprocess {file_name}")
    return df_aggregation

def get_time_interval(attack_dict):
    attack_metadata = []
    
    for attack_name, metadata in attack_dict.items():    
        if "masquerade" not in attack_name and "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Finish get time interval of {attack_name}")
            # print(count, file_name)
            
            # if attack_dict[file_name[:-4]]["injection_id"] != "XXX":
            #     print(attack_dict[file_name[:-4]]["injection_id"], int(attack_dict[file_name[:-4]]["injection_id"], 16))
            # else:
            #     print(attack_dict[file_name[:-4]]["injection_id"])
            
            # print(attack_dict[file_name[:-4]]["injection_id"], int(attack_dict[file_name[:-4]]["injection_id"], 16))
            
            # attack_metadata.append(helper_functions.get_injection_interval(df_aggregation[count], int(attack_dict[file_name[:-4]]["injection_id"], 16),
            #                                                              attack_dict[file_name[:-4]]["injection_data_str"]))
            
            # From metadata file
            attack_metadata.append([tuple(attack_dict[attack_name]["injection_interval"])])
    return attack_metadata

def mark_label(df_aggregation, attack_metadata, attack_dict):
    count = 0
    for attack_name, metadata in attack_dict.items():    
        if "masquerade" not in attack_name and "accelerator" not in attack_name and "metadata" not in attack_name:
            print(f"Index {count}: {attack_name} --- {attack_dict[attack_name]['injection_id']}")
            
            # print(attack_dict[file_name[:-4]]["injection_id"]) #, int(attack_dict[file_name[:-4]]["injection_id"], 16))#, attack_dict[file_name[:-4]]["injection_data_str"])
            
            # Add column to each attack dataframe to indicate attack (True) or non-attack (False) for each signal
            
            if attack_dict[attack_name]["injection_id"] != "XXX":
                df_aggregation[count] = helper_functions.add_actual_attack_col(df_aggregation[count], attack_metadata[count], int(attack_dict[attack_name]["injection_id"], 16), attack_dict[attack_name]["injection_data_str"])                                                      
            else:
                df_aggregation[count] = helper_functions.add_actual_attack_col(df_aggregation[count], attack_metadata[count], "XXX", attack_dict[attack_name]["injection_data_str"])
            count += 1
    return df_aggregation

def filter_attack(arr, keyword):
    sub_array = []
    for item in arr:
        if keyword in item:
            sub_array.append(item)
    return sub_array

In [71]:
attacks = ['fuzzing', 'correlated_signal', 'reverse_light', 'max_speedometer', 'max_engine_coolant_temp']

df_aggregation = []
attack_file = []

file_list = os.listdir(".")

for file_name in file_list:
    if "masquerade" not in file_name and "accelerator" not in file_name and "metadata" not in file_name:
        attack_file.append(file_name)
        
# print(f"All of fabrication and fuzzing attack: {attack_file} and length is: {len(attack_file)}")

with open("capture_metadata.json", "r") as read_file:
    attack_dict = json.load(read_file)

df_aggregation = get_all_data(attack_dict)
attack_metadata = get_time_interval(attack_dict)
df_aggregation = mark_label(df_aggregation, attack_metadata, attack_dict)

display(df_aggregation[0])
# for attack in attacks: 
#     print(f"We got {attack} at index {count}")
#     sub_attacks = filter_attack(attack_file, attack) # got list of attack ... fuzzing,...
#     print(sub_attacks)
    
#     print("Now we put to csv")
#     df_aggregation = []
#     sum_lenght = 0
#     for data in sub_attacks:
#         df_attack = helper_functions.make_can_df(data)
#         df_attack = helper_functions.add_time_diff_per_aid_col(df_attack)
#         print(df_attack.shape)
#         sum_lenght += df_attack.shape[0]
#         # print(df.dtypes)
#         df_aggregation.append(df_attack)
#         count += 1 
#     # df_res = pd.concat(df_aggregation)
#     # display(df_res)
#     print(f"Total of this boy {attack} is {sum_lenght}\n")
#     for attack in attacks: 
#         if attack in file_name and "masquerade" not in file_name and "accelerator" not in file_name and "metadata" not in file_name:
#             print(file_name)
            
#             df_attack = helper_functions.make_can_df(file_name)
#             # print(df_attack.shape)
#             # print(df.dtypes)
#             df_aggregation.append(df_attack)
#     df_res = pd.concat(df_aggregation)
#     df_aggregation = []
#     display(df_res)
# print(len(df_aggregation))

Finish preprocess correlated_signal_attack_1.log
Finish preprocess correlated_signal_attack_2.log
Finish preprocess correlated_signal_attack_3.log
Finish preprocess fuzzing_attack_1.log
Finish preprocess fuzzing_attack_2.log
Finish preprocess fuzzing_attack_3.log
Finish preprocess max_engine_coolant_temp_attack.log
Finish preprocess max_speedometer_attack_1.log
Finish preprocess max_speedometer_attack_2.log
Finish preprocess max_speedometer_attack_3.log
Finish preprocess reverse_light_off_attack_1.log
Finish preprocess reverse_light_off_attack_2.log
Finish preprocess reverse_light_off_attack_3.log
Finish preprocess reverse_light_on_attack_1.log
Finish preprocess reverse_light_on_attack_2.log
Finish preprocess reverse_light_on_attack_3.log
Finish get time interval of correlated_signal_attack_1
Finish get time interval of correlated_signal_attack_2
Finish get time interval of correlated_signal_attack_3
Finish get time interval of fuzzing_attack_1
Finish get time interval of fuzzing_attac

,time,aid,data,ori_time,time_diffs,label
3873,1.618163,6,0800006400000000,1.030000e+09,0.999845,False
6266,2.618064,6,0800006400000000,1.030000e+09,0.999901,False
8655,3.617806,6,0800006400000000,1.030000e+09,0.999742,False
11049,4.617810,6,0800006400000000,1.030000e+09,1.000004,False
13441,5.618164,6,0800006400000000,1.030000e+09,1.000354,False
...,...,...,...,...,...,...
80160,32.636582,1788,00000738D2B85800,1.030000e+09,0.099969,False
80402,32.736450,1788,0000073732B86000,1.030000e+09,0.099868,False
80641,32.836612,1788,0000073892B86800,1.030000e+09,0.100162,False
80874,32.936532,1788,00000737B2B87000,1.030000e+09,0.099920,False


In [72]:
count = 0
outdir = '/mnt/hdd2/transformer-entropy-ids/road/processed/'

for attack_name, metadata in attack_dict.items():
    if "masquerade" not in attack_name and "accelerator" not in attack_name and "metadata" not in attack_name:
        print(f"Save into {attack_name}_dataset.csv")
        foutput = '{}/{}_dataset.csv'.format(outdir, attack_name)
        df_aggregation[count].to_csv(foutput, index=False)
        count += 1

Save into correlated_signal_attack_1_dataset.csv
Save into correlated_signal_attack_2_dataset.csv
Save into correlated_signal_attack_3_dataset.csv
Save into fuzzing_attack_1_dataset.csv
Save into fuzzing_attack_2_dataset.csv
Save into fuzzing_attack_3_dataset.csv
Save into max_engine_coolant_temp_attack_dataset.csv
Save into max_speedometer_attack_1_dataset.csv
Save into max_speedometer_attack_2_dataset.csv
Save into max_speedometer_attack_3_dataset.csv
Save into reverse_light_off_attack_1_dataset.csv
Save into reverse_light_off_attack_2_dataset.csv
Save into reverse_light_off_attack_3_dataset.csv
Save into reverse_light_on_attack_1_dataset.csv
Save into reverse_light_on_attack_2_dataset.csv
Save into reverse_light_on_attack_3_dataset.csv
